In [37]:
import pandas as pd 
import os
from sklearn.model_selection import train_test_split
## for cleaning 
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack


In [38]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [39]:
import zipfile
import os

def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)

# Example usage:
zip_file_path = 'archive (13).zip'
extract_to_path = 'ziped'

# Create the destination directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

unzip_file(zip_file_path, extract_to_path)


In [40]:
true=pd.read_csv("ziped\True.csv")
fake=pd.read_csv("ziped\Fake.csv")

In [41]:
true["f/t"]="1"
fake["f/t"]="0"

In [42]:
true.shape,fake.shape

((21417, 5), (23481, 5))

In [43]:
df=pd.concat([true,fake])

In [44]:
df=df.sample(frac=1).reset_index(drop=True)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   f/t      44898 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [46]:
df=df.drop('date',axis=1)
df=df.drop('subject',axis=1)
df['title']=df['title'].astype(str)
df['text']=df['text'].astype(str)

In [47]:

def clean(text):

    text = text.lower()
    text = re.sub(r'https?://S+ | www\.\S+', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub('\[!.*?\]', '', text)
    word=""
    for i in text : 
        if i not in string.punctuation :
            word+=i
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_sentence = ' '.join(tokens) 
    
    return preprocessed_sentence
clean(df.loc[1]['text'])
    

'trump pick press secretary , noted numbnut sean spicer , appeared abc week discus , well , various thing . one point , asked whether russia actually try rig election , decided deflect instead actually give u real opinion . however , deflection plain insane . despite fact hillary lost election , decided say : part conversation . talking influence election ? talking hillary clinton getting debate question ahead time ? pretty valid attempt influence election . well , first , lost , spicer even bringing ? likely excuse russia meddling election , along fact trump closeness russia look really bad . look terrible way make look pretty without deflecting . , yeah , sure , russia bad , hillary worse , blah blah blah punish ! yes , spicer said , : going start talking side ? : hillary clinton influence election ? punished way ? please . someone stop people embarrassing whole country . watch whole ridiculous segment : featured image via screen capture embedded video'

In [48]:
### apply clean
df["title"]=df["title"].apply(clean)
df["text"]=df["text"].apply(clean)

In [ ]:
#save the df bcz cleaning takes too much time
df.to_csv("df.csv",index=False)

In [49]:
#split the data 
labels=df["f/t"]
features=df.drop("f/t",axis=1)
y_train,y_test,x_train,x_test=train_test_split(labels,features,test_size=0.2,random_state=42)


In [50]:
from joblib import dump, load
vector1 = TfidfVectorizer()
vector2 = TfidfVectorizer()

# Vectorize each column separately
x_train1 = vector1.fit_transform(x_train['title'])
x_train2 = vector2.fit_transform(x_train['text'])

# Concatenate the vectors horizontally
x_train_vectorized = hstack([x_train1, x_train2])

# Repeat the process for the test set
x_test1 = vector1.transform(x_test['title'])
x_test2 = vector2.transform(x_test['text'])
x_test_vectorized = hstack([x_test1, x_test2])
dump(vector1, 'tfidf_vectorizer1.joblib')
dump(vector2, 'tfidf_vectorizer2.joblib')

['tfidf_vectorizer2.joblib']

In [52]:
#model selection 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
#model=SVC()
#model.fit(x_train_vectorized,y_train)
#y_pred=model.predict(x_test_vectorized)
#accc=accuracy_score(y_test,y_pred)
#accc : 0.9947661469933184

In [53]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(x_train_vectorized,y_train)
y_pred=model.predict(x_test_vectorized)
accc=accuracy_score(y_test,y_pred)
accc= 0.9968819599109131

In [54]:
from joblib import dump, load
dump(model, 'model.joblib') 

['model.joblib']

In [ ]:
load_model = load('filename.joblib') 